In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pytorch-tabnet==3.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import pandas as pd
import torch

from torch import nn
from pytorch_tabnet.tab_model  import TabNetRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [4]:
covid = pd.read_csv('/content/drive/MyDrive/covid/covid.csv',encoding = 'cp949')

In [5]:
kor_df = covid[covid['iso_code']=='KOR']
kor_df

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
156083,KOR,Asia,South Korea,2020-01-22,1.0,NaN,NaN,NaN,NaN,NaN,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN
156084,KOR,Asia,South Korea,2020-01-23,1.0,0.0,NaN,NaN,NaN,NaN,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN
156085,KOR,Asia,South Korea,2020-01-24,2.0,1.0,NaN,NaN,NaN,NaN,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN
156086,KOR,Asia,South Korea,2020-01-25,2.0,0.0,NaN,NaN,NaN,NaN,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN
156087,KOR,Asia,South Korea,2020-01-26,3.0,1.0,NaN,NaN,NaN,NaN,...,6.2,40.9,NaN,12.27,83.03,0.916,43.4,0.17,2.47,0.845918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156923,KOR,Asia,South Korea,2022-05-11,17694677.0,35883.0,36658.429,23554.0,63.0,56.571,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN
156924,KOR,Asia,South Korea,2022-05-12,17727086.0,32409.0,37472.000,23606.0,52.0,57.143,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN
156925,KOR,Asia,South Korea,2022-05-13,17756627.0,29541.0,36041.857,23661.0,55.0,53.143,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN
156926,KOR,Asia,South Korea,2022-05-14,17782061.0,25434.0,33951.857,23709.0,48.0,49.857,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN


In [6]:
cols = ['iso_code', 'continent', 'location', 'date','tests_units']
kor_df.drop(cols, axis=1, inplace=True)
kor_df

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
156083,1.0,NaN,NaN,NaN,NaN,NaN,0.019,NaN,NaN,NaN,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN
156084,1.0,0.0,NaN,NaN,NaN,NaN,0.019,0.000,NaN,NaN,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN
156085,2.0,1.0,NaN,NaN,NaN,NaN,0.039,0.019,NaN,NaN,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN
156086,2.0,0.0,NaN,NaN,NaN,NaN,0.039,0.000,NaN,NaN,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN
156087,3.0,1.0,NaN,NaN,NaN,NaN,0.058,0.019,NaN,NaN,...,6.2,40.9,NaN,12.27,83.03,0.916,43.4,0.17,2.47,0.845918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156923,17694677.0,35883.0,36658.429,23554.0,63.0,56.571,344890.625,699.403,714.517,459.096,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN
156924,17727086.0,32409.0,37472.000,23606.0,52.0,57.143,345522.316,631.691,730.375,460.109,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN
156925,17756627.0,29541.0,36041.857,23661.0,55.0,53.143,346098.106,575.790,702.499,461.181,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN
156926,17782061.0,25434.0,33951.857,23709.0,48.0,49.857,346593.845,495.739,661.763,462.117,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN


In [7]:
kor = kor_df.fillna(0)
kor

,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
156083,1.0,0.0,0.000,0.0,0.0,0.000,0.019,0.000,0.000,0.000,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
156084,1.0,0.0,0.000,0.0,0.0,0.000,0.019,0.000,0.000,0.000,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
156085,2.0,1.0,0.000,0.0,0.0,0.000,0.039,0.019,0.000,0.000,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
156086,2.0,0.0,0.000,0.0,0.0,0.000,0.039,0.000,0.000,0.000,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
156087,3.0,1.0,0.000,0.0,0.0,0.000,0.058,0.019,0.000,0.000,...,6.2,40.9,0.0,12.27,83.03,0.916,43.4,0.17,2.47,0.845918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156923,17694677.0,35883.0,36658.429,23554.0,63.0,56.571,344890.625,699.403,714.517,459.096,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
156924,17727086.0,32409.0,37472.000,23606.0,52.0,57.143,345522.316,631.691,730.375,460.109,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
156925,17756627.0,29541.0,36041.857,23661.0,55.0,53.143,346098.106,575.790,702.499,461.181,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000
156926,17782061.0,25434.0,33951.857,23709.0,48.0,49.857,346593.845,495.739,661.763,462.117,...,6.2,40.9,0.0,12.27,83.03,0.916,0.0,0.00,0.00,0.000000


In [8]:
kor.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 845 entries, 156083 to 156927
Data columns (total 62 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   total_cases                                 845 non-null    float64
 1   new_cases                                   845 non-null    float64
 2   new_cases_smoothed                          845 non-null    float64
 3   total_deaths                                845 non-null    float64
 4   new_deaths                                  845 non-null    float64
 5   new_deaths_smoothed                         845 non-null    float64
 6   total_cases_per_million                     845 non-null    float64
 7   new_cases_per_million                       845 non-null    float64
 8   new_cases_smoothed_per_million              845 non-null    float64
 9   total_deaths_per_million                    845 non-null    float64
 10  new_de

In [9]:
X = kor.drop('new_cases', axis=1)
y = kor['new_cases']

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [11]:
train_X = X_train.to_numpy()
train_y = y_train.to_numpy().reshape(-1,1)
# val_X = X_val.to_numpy()
# val_y = y_val.to_numpy().reshape(-1,1)
test_X = X_test.to_numpy()

In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaler = scaler.fit_transform(train_X)
y_train_scaler = scaler.fit_transform(train_y)
X_test_scaler = scaler.fit_transform(test_X)

In [13]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)
scores  = []

for trn_idx,val_idx in kf.split(train_X):
    X_trn = X_train_scaler[trn_idx]
    y_trn = y_train_scaler[trn_idx]
    X_val = X_train_scaler[val_idx]
    y_val = y_train_scaler[val_idx]

    model = TabNetRegressor(verbose = 1,seed = 42,optimizer_fn=torch.optim.AdamW)
    model.fit(X_train=X_trn, y_train=y_trn,
              eval_set=[(X_val, y_val)],
            patience=100, max_epochs=2000,
              eval_metric=['rmse'])
    scores.append(model.best_cost)
    
    pred = model.predict(X_test_scaler)

Device used : cpu
epoch 0  | loss: 0.31419 | val_0_rmse: 0.06968 |  0:00:00s
epoch 1  | loss: 0.2634  | val_0_rmse: 0.11754 |  0:00:00s
epoch 2  | loss: 0.23976 | val_0_rmse: 0.11103 |  0:00:00s
epoch 3  | loss: 0.12593 | val_0_rmse: 0.11164 |  0:00:00s
epoch 4  | loss: 0.09105 | val_0_rmse: 0.11112 |  0:00:00s
epoch 5  | loss: 0.08319 | val_0_rmse: 0.112   |  0:00:00s
epoch 6  | loss: 0.06647 | val_0_rmse: 0.11404 |  0:00:00s
epoch 7  | loss: 0.05834 | val_0_rmse: 0.11414 |  0:00:00s
epoch 8  | loss: 0.03854 | val_0_rmse: 0.11589 |  0:00:00s
epoch 9  | loss: 0.02762 | val_0_rmse: 0.11663 |  0:00:00s
epoch 10 | loss: 0.02735 | val_0_rmse: 0.11709 |  0:00:00s
epoch 11 | loss: 0.02588 | val_0_rmse: 0.11754 |  0:00:00s
epoch 12 | loss: 0.02839 | val_0_rmse: 0.11849 |  0:00:00s
epoch 13 | loss: 0.0199  | val_0_rmse: 0.11929 |  0:00:01s
epoch 14 | loss: 0.01417 | val_0_rmse: 0.12019 |  0:00:01s
epoch 15 | loss: 0.01657 | val_0_rmse: 0.1213  |  0:00:01s
epoch 16 | loss: 0.01725 | val_0_rmse:

In [14]:
test_y = y_test.to_numpy().reshape(-1,1)

y_test = scaler.fit_transform(test_y)
y_test

array([[9.02920731e-04],
       [4.29732327e-04],
       [1.62718870e-03],
       [1.05582175e-03],
       [1.01558464e-03],
       [1.08801143e-03],
       [3.39118357e-03],
       [8.70087250e-03],
       [1.49886451e-01],
       [3.29783347e-03],
       [0.00000000e+00],
       [3.31569875e-02],
       [1.69800601e-03],
       [6.83271180e-01],
       [1.08962092e-03],
       [1.00000000e+00],
       [3.70181405e-05],
       [1.30384329e-01],
       [2.01507443e-03],
       [4.34560780e-05],
       [6.71154982e-04],
       [1.89919156e-04],
       [1.43244109e-04],
       [1.07030711e-03],
       [6.27537956e-03],
       [3.23248841e-02],
       [5.12459823e-03],
       [7.88647341e-05],
       [3.30409437e-01],
       [2.09297347e-02],
       [7.24267966e-05],
       [2.30639110e-03],
       [1.22674899e-02],
       [1.46181418e-01],
       [5.71366951e-04],
       [9.20222688e-02],
       [0.00000000e+00],
       [2.18889874e-04],
       [5.15034998e-05],
       [1.68191117e-03],


In [21]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
MAE =  mean_absolute_error(y_test, pred)

MSE = mean_squared_error(y_test, pred)

RMSE = np.sqrt(MSE)

R2 = r2_score(y_test, pred)

cv = np.std(scores)/np.mean(scores)

print("MAE 점수 : %.5f" % MAE)
print("MSE 점수 : %.5f" % MSE)
print("RMSE 점수 : %.5f" % RMSE)
print("R2 점수 : %.5f" % R2)
print('CV계수 : %.5f' % cv)

MAE 점수 : 0.02123
MSE 점수 : 0.00135
RMSE 점수 : 0.03677
R2 점수 : 0.92477
CV계수 : 0.94447
